# This notebook 

- convert DBF from GIS to csv 

- extracts zipcode for each intersection in Chicago 

- calculate the intersection density for each zipcode.

Package:

- `dbfread`

- `pandas`

- `scipy`


# Data resources : 

- [How to convert dbf to csv](https://stackoverflow.com/questions/32772447/way-to-convert-dbf-to-csv-in-python)


- [U.S. Street Network Shapefiles, Node/Edge Lists, and GraphML Files](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/CUWWYJ)

In [29]:
# Necessary imports
from dbfread import DBF
import pandas as pd
import csv
import numpy as np
import math
from scipy import spatial
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# I. Helper function

In [30]:
def to_Cartesian(lat, lon):
    '''
    function to convert latitude and longitude to 3D cartesian coordinates
    '''
    R = 6371 # radius of the Earth in kilometers

    x = R * math.cos(lat) * math.cos(lon)
    y = R * math.cos(lat) * math.sin(lon)
    z = R * math.sin(lat)
    return x, y, z

def deg2rad(degree):
    '''
    function to convert degree to radian
    '''
    rad = degree * 2*np.pi / 360
    return rad

def rad2deg(rad):
    '''
    function to convert radian to degree
    '''
    degree = rad/2/np.pi * 360
    return degree

def distToKM(x):
    '''
    function to convert cartesian distance to real distance in km
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(deg2rad(x/(2*R))) # compute the angle of the isosceles triangle
    dist = 2*R*math.sin(gamma/2) # compute the side of the triangle
    return dist

def kmToDIST(x):
    '''
    function to convert real distance in km to cartesian distance 
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(x/2./R) 
    
    dist = 2*R*rad2deg(sin(gamma / 2.))
    return dist

In [31]:
def interfer_zipcode_from_distance(df, lat_ref, lng_ref , n_closest=5):
    '''
    function to estimate the zipcode of each lat/lgn in df by calculating 
    it's Euclide distance to every reference lat/lgn in lat_ref/lgn_ref,
    return top 5 closest ref_lat/ref_lgn 
    '''
    
    lats_1d = df['lat'].values
    lons_1d = df['lng'].values

    # convert the grid points and reference points to cartesian coordinates
    x, y, z = zip(*map(to_Cartesian, lats_1d, lons_1d))
    x_ref, y_ref, z_ref = to_Cartesian(lat_ref, lng_ref)
    
    # create the KD-tree using the 3D cartesian coordinates
    coordinates = list(zip(x, y, z))
    tree = spatial.cKDTree(coordinates)

    # get the cartesian distances from the 10 closest points
    dist, ix = tree.query((x_ref, y_ref, z_ref), n_closest)

    # print out the real distances in km
    #print(list(map(distToKM, dist)))
    
    #return dist,ix
    return df['zipcode'].iloc[ix[0]]

In [32]:
def dbf_to_csv(dbf_table_pth):
    '''
    function to covert dbf to csv
    Input : dbf
    Output : csv, same name, same path, except extension
    '''

    csv_fn = dbf_table_pth[:-4]+ ".csv" #Set the csv file name
    table = DBF(dbf_table_pth)# table variable is a DBF object
    with open(csv_fn, 'w', newline = '') as f:# create a csv file, fill it with dbf content
        writer = csv.writer(f)
        writer.writerow(table.field_names)# write the column name
        for record in table:# write the rows
            writer.writerow(list(record.values()))
    return csv_fn# return the csv name


In [33]:
# how to read dbf file
dbf_table_pth = './gis_dataset/ZIPintersections/ZIPintersections.dbf'

table = DBF(dbf_table_pth, load=True)
#df = table.to_dataframe()
#for record in table:
#    print(record)

dbf_to_csv(dbf_table_pth)

'./gis_dataset/ZIPintersections/ZIPintersections.csv'

# II. Extract road intersection from GIS database 

## 1. Extract the number of intersections in each zipcode

In [34]:
dbf_table_pth = './gis_dataset/ZIPintersections/ZIPintersections.dbf'
csv_path = dbf_to_csv(dbf_table_pth)
df = pd.read_csv(csv_path)

In [35]:
df = df.rename(columns={'zip' : 'zipcode', 'Count_' : 'intersection_count'})

print('number of row :',len(df))
print('number of zipcodes are :',len(set(df.zipcode)))
print('There are total {} intersections in Chicago'.format(df['intersection_count'].sum())) 
print(df.columns)
df.head()

number of row : 61
number of zipcodes are : 59
There are total 28335 intersections in Chicago
Index(['FID_1', 'objectid', 'shape_area', 'shape_len', 'zipcode',
       'intersection_count', 'Sum_osmid', 'Sum_x', 'Sum_y'],
      dtype='object')


,FID_1,objectid,shape_area,shape_len,zipcode,intersection_count,Sum_osmid,Sum_x,Sum_y
0,0,33.0,1.060523e+08,42720.044406,60647,699,2.018352e+11,-61302.665277,29302.793967
1,1,34.0,1.274761e+08,48103.782721,60639,612,1.791981e+11,-53706.591818,25654.888800
2,2,35.0,4.506904e+07,27288.609612,60707,172,5.470885e+10,-15101.150940,7210.159568
3,3,36.0,7.085383e+07,42527.989679,60622,419,1.230189e+11,-36738.913588,17557.220936
4,4,37.0,9.903962e+07,47970.140153,60651,504,1.395923e+11,-44220.573629,21118.735909


In [36]:
# pickling the number of intersections in each zipcode 
df[['zipcode', 'intersection_count']].to_pickle('./dataset/intersections_per_zipcode.pkl')
df.head(3)

,FID_1,objectid,shape_area,shape_len,zipcode,intersection_count,Sum_osmid,Sum_x,Sum_y
0,0,33.0,1.060523e+08,42720.044406,60647,699,2.018352e+11,-61302.665277,29302.793967
1,1,34.0,1.274761e+08,48103.782721,60639,612,1.791981e+11,-53706.591818,25654.888800
2,2,35.0,4.506904e+07,27288.609612,60707,172,5.470885e+10,-15101.150940,7210.159568


## 2. Extract the zipcode  for each intersection

In [37]:
!ls gis_dataset/intersections_with_zip

intersections_with_ZIP.csv intersections_with_ZIP.shp
intersections_with_ZIP.dbf intersections_with_ZIP.shx
intersections_with_ZIP.prj


In [38]:
dbf_table_pth = './gis_dataset/intersections_with_zip/intersections_with_ZIP.dbf'
csv_path = dbf_to_csv(dbf_table_pth)
df = pd.read_csv(csv_path)
df.head(3)

,FID_1,osmid,x,y,ref,highway,FID_2,objectid,shape_area,shape_len,zip
0,52,26703646,-87.694086,41.930899,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0
1,53,26703648,-87.691585,41.929645,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0
2,54,26703650,-87.687291,41.927358,47A,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647.0


In [39]:
df = df.rename(columns={'zip' : 'zipcode', 'y' : 'lat', 'x' : 'lgn'})

print('number of zipcodes are :',len(set(df.zipcode)))
print(df.columns)
df.head(3)

number of zipcodes are : 100
Index(['FID_1', 'osmid', 'lgn', 'lat', 'ref', 'highway', 'FID_2', 'objectid',
       'shape_area', 'shape_len', 'zipcode'],
      dtype='object')


,FID_1,osmid,lgn,lat,ref,highway,FID_2,objectid,shape_area,shape_len,zipcode
0,52,26703646,-87.694086,41.930899,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0
1,53,26703648,-87.691585,41.929645,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0
2,54,26703650,-87.687291,41.927358,47A,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647.0


In [40]:
set(df['highway'])

{'bus_stop',
 'construction',
 'crossing',
 'crossing;traffic_signals',
 'mini_roundabout',
 'motorway_junction',
 nan,
 'stop',
 'traffic_signals',
 'turning_circle'}

###  Check zipcode with Nan values

In [41]:
# check nan zipcode 
df[df['zipcode'].isna()][['lat', 'lgn', 'zipcode']]

,lat,lgn,zipcode
28331,42.004319,-87.894079,NaN
28332,41.970665,-87.816953,NaN
28333,41.950724,-87.892766,NaN
28334,41.663246,-87.641437,NaN
28335,41.694857,-87.720479,NaN
28336,41.686136,-87.700620,NaN
28337,41.706184,-87.701333,NaN
28338,42.000853,-87.806757,NaN
28339,41.982807,-87.732107,NaN
28340,41.799420,-87.796640,NaN


### Estimate the zipcode from its lat/lgn

In [42]:
# create df containing the missing zipcode 
df_no_zipcode = df[df['zipcode'].isna()][['lat', 'lgn']]


# load the centroid coordinate of reference zipcode
df_centroid_zipcode = pd.read_pickle('./dataset/chicago_zipcode.pkl')


# find the missing zipcode from its distance to referred location
df_no_zipcode['zipcode'] = df_no_zipcode[['lat', 'lgn']].apply(lambda x : \
                                     interfer_zipcode_from_distance( df_centroid_zipcode , x[0], x[1]), axis = 1)

df_no_zipcode.head(3)

,lat,lgn,zipcode
28331,42.004319,-87.894079,60018
28332,41.970665,-87.816953,60656
28333,41.950724,-87.892766,60131


In [43]:
# merge new zipcode with old df
df = pd.merge(df, df_no_zipcode, how='outer', on = ['lat', 'lgn'])
df

,FID_1,osmid,lgn,lat,ref,highway,FID_2,objectid,shape_area,shape_len,zipcode_x,zipcode_y
0,52,26703646,-87.694086,41.930899,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
1,53,26703648,-87.691585,41.929645,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
2,54,26703650,-87.687291,41.927358,47A,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
3,62,26704168,-87.678438,41.922676,47A,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
4,63,26704169,-87.679930,41.923176,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
5,64,26704178,-87.690595,41.929581,46B,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
6,346,35034247,-87.683685,41.924956,NaN,traffic_signals,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
7,348,35034253,-87.677766,41.921390,NaN,traffic_signals,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
8,358,43510489,-87.684937,41.915225,NaN,stop,0,33.0,1.060523e+08,42720.044406,60647.0,NaN
9,359,43512551,-87.689186,41.917923,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647.0,NaN


In [44]:
# fill Nan zipcode in zipcode_x with new zipcode from zipcode_y
mask = df['zipcode_x'].isna()
df['zipcode_x'][mask] = df['zipcode_y'][mask] 

# rename column
df = df.rename(columns={'zipcode_x' : 'zipcode'})

# convert zipcode into integer
df['zipcode'] = df['zipcode'].astype(int)
print(df.columns)
df.head(3)

Index(['FID_1', 'osmid', 'lgn', 'lat', 'ref', 'highway', 'FID_2', 'objectid',
       'shape_area', 'shape_len', 'zipcode', 'zipcode_y'],
      dtype='object')


/Users/ducvu/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,FID_1,osmid,lgn,lat,ref,highway,FID_2,objectid,shape_area,shape_len,zipcode,zipcode_y
0,52,26703646,-87.694086,41.930899,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647,NaN
1,53,26703648,-87.691585,41.929645,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647,NaN
2,54,26703650,-87.687291,41.927358,47A,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647,NaN


In [45]:
# pickling df 
df[['zipcode', 'lgn', 'lat', 'highway']].to_pickle('./dataset/intersections_with_zipcode.pkl')
df.head(3)

,FID_1,osmid,lgn,lat,ref,highway,FID_2,objectid,shape_area,shape_len,zipcode,zipcode_y
0,52,26703646,-87.694086,41.930899,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647,NaN
1,53,26703648,-87.691585,41.929645,NaN,NaN,0,33.0,1.060523e+08,42720.044406,60647,NaN
2,54,26703650,-87.687291,41.927358,47A,motorway_junction,0,33.0,1.060523e+08,42720.044406,60647,NaN


## 3. Extract the cross roads  for each intersection

In [46]:
!ls gis_dataset/intersectionwithnames

intersectionwithnames.csv intersectionwithnames.sbn intersectionwithnames.shx
intersectionwithnames.dbf intersectionwithnames.sbx intersectionwithnames.xml
intersectionwithnames.prj intersectionwithnames.shp


In [47]:
dbf_table_pth = './gis_dataset/intersectionwithnames/intersectionwithnames.dbf'
csv_path = dbf_to_csv(dbf_table_pth)
df = pd.read_csv(csv_path)

print(len(df))
df = df.rename(columns={'y' : 'lat', 'x' : 'lgn'})
#print('number of zipcodes are :',len(set(df.zipcode)))
print(df.columns)
df.head(3)

/Users/ducvu/anaconda3/envs/metis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


95661
Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'JOIN_FID', 'osmid', 'lgn',
       'lat', 'ref', 'highway', 'FNODE_ID', 'TNODE_ID', 'TRANS_ID', 'PRE_DIR',
       'STREET_NAM', 'STREET_TYP', 'SUF_DIR', 'STREETNAME', 'L_F_ADD',
       'L_T_ADD', 'R_F_ADD', 'R_T_ADD', 'LOGICLF', 'LOGICLT', 'LOGICRF',
       'LOGICRT', 'CLASS', 'STATUS', 'STATUS_DAT', 'TIERED', 'ONEWAY_DIR',
       'DIR_TRAVEL', 'EWNS', 'L_PARITY', 'R_PARITY', 'F_ZLEV', 'T_ZLEV',
       'L_FIPS', 'R_FIPS', 'R_ZIP', 'L_ZIP', 'R_CENSUSBL', 'L_CENSUSBL',
       'F_CROSS', 'F_CROSS_ST', 'T_CROSS', 'T_CROSS_ST', 'LENGTH', 'EDIT_DATE',
       'EDIT_TYPE', 'FLAG_STRIN', 'EWNS_DIR', 'EWNS_COORD', 'CREATE_USE',
       'CREATE_TIM', 'UPDATE_USE', 'UPDATE_TIM'],
      dtype='object')


,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,osmid,lgn,lat,ref,highway,FNODE_ID,...,LENGTH,EDIT_DATE,EDIT_TYPE,FLAG_STRIN,EWNS_DIR,EWNS_COORD,CREATE_USE,CREATE_TIM,UPDATE_USE,UPDATE_TIM
0,1,1,0,28367,701663.0,-87.844405,41.983874,79A,motorway_junction,2983.0,...,1464.880741,0.0,NaN,NaN,NaN,0.0,EXISTING,1999-01-01,EXISTING,1999-01-01
1,2,1,1,4723,701665.0,-87.847486,41.983985,78,motorway_junction,17567.0,...,2370.230017,0.0,NaN,DUPFROMTO,NaN,0.0,EXISTING,1999-01-01,EXISTING,1999-01-01
2,3,1,1,33907,701665.0,-87.847486,41.983985,78,motorway_junction,2982.0,...,2380.050458,20100803.0,Street Name Change,NaN,NaN,0.0,EXISTING,1999-01-01,ds06027,2010-08-03


In [48]:
df.iloc[12222]

OBJECTID                      12223
Join_Count                        1
TARGET_FID                     3553
JOIN_FID                      34177
osmid                   2.61128e+08
lgn                        -87.6876
lat                         41.9658
ref                             NaN
highway                         NaN
FNODE_ID                      35010
TNODE_ID                      33166
TRANS_ID                     102347
PRE_DIR                           W
STREET_NAM                 EASTWOOD
STREET_TYP                      AVE
SUF_DIR                         NaN
STREETNAME                     2158
L_F_ADD                        2301
L_T_ADD                        2317
R_F_ADD                        2300
R_T_ADD                        2316
LOGICLF                        2301
LOGICLT                        2319
LOGICRF                        2300
LOGICRT                        2316
CLASS                             4
STATUS                            N
STATUS_DAT               199

In [49]:
df[['lat', 'lgn', 'STREET_NAM', 'STREET_TYP', 'F_CROSS', 'F_CROSS_ST', 'T_CROSS', 'T_CROSS_ST']] 

,lat,lgn,STREET_NAM,STREET_TYP,F_CROSS,F_CROSS_ST,T_CROSS,T_CROSS_ST
0,41.983874,-87.844405,KENNEDY,EXPY,8599|W|||,0.0,8800|W|IB NORTHWEST|TOLL|,2321.0
1,41.983985,-87.847486,OB NORTHWEST,TOLL,5800|N|EAST RIVER|RD|,470.0,9131|W|||,0.0
2,41.983985,-87.847486,I190,EXPY,8600|W|KENNEDY|EXPY|,2350.0,9131|W|||,0.0
3,41.984697,-87.831466,KENNEDY CUMBERLAND AV,XR,8200|W|KENNEDY|EXPY|,2350.0,8319|W|HIGGINS|RD|,2289.0
4,41.984697,-87.831466,KENNEDY,EXPY,8200|W|KENNEDY CUMBERLAND AV|XR|,2348.0,5800|N|KENNEDY CUMBERLAND AV|ER|,591.0
5,41.983432,-87.817823,KENNEDY,EXPY,7632|W|KENNEDY HIGGINS RD|XR|,2354.0,7901|W|||,410.0
6,41.983432,-87.817823,KENNEDY,EXPY,7599|W|||,0.0,7632|W|KENNEDY HIGGINS RD|XR|,2354.0
7,41.983432,-87.817823,KENNEDY HIGGINS RD,XR,7632|W|KENNEDY|EXPY|,2350.0,7800|W|HIGGINS|AVE|,2288.0
8,41.983129,-87.817597,KENNEDY,EXPY,7599|W|||,0.0,7634|W|CANFIELD AV JFK|ER|,2083.0
9,41.900902,-87.660921,KENNEDY,EXPY,1035|N|KENNEDY DIVISION ST|ER|,597.0,1101|N|||,0.0
